<a href="https://colab.research.google.com/github/pablocedillos28/Grothendieck-polynomials/blob/main/Type_A_Grothendieck_Polynomials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Grothendieck polynomial calculator (degenerate Hecke algebra)

from sympy import symbols, expand
import itertools

# --------------------------
# Symbols
# --------------------------
beta = symbols('beta')   # symbolic β

# --------------------------
# Permutation helpers
# --------------------------
def identity_perm(n):
    return tuple(range(1, n+1))

def apply_simple_reflection_on_perm_left(i, perm):
    """
    Left-multiply permutation 'perm' by simple reflection s_i (1-based index i).
    That is compute s_i ∘ perm (apply perm, then swap images at i and i+1).
    We represent permutations as tuples p of length n with values 1..n,
    where p[j-1] is the image of j.
    Left multiplication by s_i gives new tuple r with:
       r[j-1] = s_i( p[j-1] ) = p[j-1] but swap values equal to i and i+1
    Simpler: swap positions i and i+1 of the tuple perm.
    """
    perm = list(perm)
    perm[i-1], perm[i] = perm[i], perm[i-1]
    return tuple(perm)

def inversion_number(p):
    """Number of inversions of permutation tuple p (length)."""
    n = len(p)
    return sum(1 for i in range(n) for j in range(i+1, n) if p[i] > p[j])

# --------------------------
# Reduced word (one choice) for permutation p
# We'll produce one reduced word (list of generator indices) via bubble-sort-style moves.
# This is only used to express u_p as a product of generators u_{a1}...u_{ar}.
# --------------------------
def reduced_word_for_perm(p):
    """
    Produce one reduced word for permutation p using adjacent swaps (bubble-sort).
    Returns a list of generator indices (1-based).
    """
    arr = list(p)
    n = len(arr)
    word = []
    # We will bubble each value to its correct place using adjacent swaps.
    # This produces a reduced decomposition (not necessarily unique, but valid).
    target = list(range(1, n+1))
    # Work on a copy and perform swaps until sorted
    a = arr.copy()
    # We want to get from a to identity by swapping adjacent entries where left > right,
    # but bubble-sort on the values gives a sequence of adjacent transpositions that
    # realize p^{-1}. For our purpose, any reduced decomposition for p is fine.
    # We'll do insertion-like procedure: for value v from 1..n, move it to position v-1.
    for v in range(1, n+1):
        pos = a.index(v)
        while pos > v-1:
            # swap positions pos-1 and pos (these correspond to generator index pos)
            # generator index is pos (1-based for swapping entries pos and pos+1)
            i = pos  # 1-based
            # perform swap
            a[pos-1], a[pos] = a[pos], a[pos-1]
            word.append(i)
            pos -= 1
    # The produced word is a reduced word for p^{-1}; reverse to get word for p.
    word = list(reversed(word))
    return word

# --------------------------
# Left multiplication of basis element u_w by a single generator u_i
# using the degenerate Hecke rule:
#   u_i u_w = u_{s_i w}  if l(s_i w) = l(w) + 1
#           = beta * u_w    if l(s_i w) = l(w) - 1
# (We return a dict mapping permutation -> coeff)
# --------------------------
def left_multiply_by_generator_on_basis(i, w):
    """
    i: 1-based generator index
    w: permutation tuple
    returns: dict { perm_tuple : sympy_coeff }
    """
    s_i_w = apply_simple_reflection_on_perm_left(i, w)
    l_w = inversion_number(w)
    l_si_w = inversion_number(s_i_w)
    if l_si_w == l_w + 1:
        return { s_i_w : 1 }
    else:
        # l_si_w == l_w - 1
        return { w : beta }

# --------------------------
# Multiply basis element u_p by basis element u_w
# If p corresponds to permutation p, express u_p as product of generators
# using a reduced word for p, then apply left-multiplication sequentially.
# Returns dict mapping permutations -> sympy coefficients.
# --------------------------
def multiply_basis_elements(p_perm, w_perm):
    """
    Multiply u_p * u_w, where p_perm and w_perm are permutations (tuples).
    Returns dict {perm: coeff}
    """
    # get reduced word for p (one choice)
    word = reduced_word_for_perm(p_perm)
    # start with single term u_w
    result = { w_perm : 1 }
    # apply generators in order (leftmost first)
    for gen in word:
        new_res = {}
        for perm, coeff in result.items():
            mul = left_multiply_by_generator_on_basis(gen, perm)
            for perm2, coeff2 in mul.items():
                new_res[perm2] = new_res.get(perm2, 0) + coeff * coeff2
        result = new_res
    return result

# --------------------------
# Multiply general algebra elements represented as dict {perm: coeff}
# where coeff are sympy expressions (in x's and beta).
# Multiplication is bilinear and uses multiply_basis_elements for basis multiplication.
# --------------------------
def multiply_elements(A, B):
    """
    A, B: dict { permutation tuple : sympy-coeff }
    returns: dict { permutation : sympy-expression }
    """
    C = {}
    for p, c_p in A.items():
        for q, c_q in B.items():
            # multiply basis u_p * u_q -> sum over r with coeff α_{p,q}^r
            prod = multiply_basis_elements(p, q)  # dict
            for r, coeff_r in prod.items():
                C[r] = C.get(r, 0) + c_p * c_q * coeff_r
    return C

# --------------------------
# Build A_i(t) = (1 + t u_{n-1}) ... (1 + t u_i)
# as dict {perm: coeff}
# --------------------------
def Ai(i, t, n):
    A = { identity_perm(n) : 1 }
    # multiply from right by factors (1 + t u_j) for j = n-1 down to i
    for j in range(n-1, i-1, -1):
        # build factor (1 + t u_j) as an algebra element
        # 1 -> identity_perm, u_j -> basis element with permutation s_j
        s_j = apply_simple_reflection_on_perm_left(j, identity_perm(n))
        factor = { identity_perm(n) : 1, s_j : t }
        A = multiply_elements(A, factor)
    return A

# --------------------------
# Full product A_1(x1) ... A_{n-1}(x_{n-1})
# --------------------------
def full_A(x_syms, n):
    A_total = { identity_perm(n) : 1 }
    for i in range(1, n):
        A_i = Ai(i, x_syms[i-1], n)
        A_total = multiply_elements(A_total, A_i)
    return A_total

# --------------------------
# Grothendieck polynomial extraction
# --------------------------
def grothendieck_polynomial(w, n):
    """
    Compute G_w(X) for permutation w (tuple).
    Returns a sympy expression in x1..x_{n-1} and beta.
    """
    x_syms = symbols('x1:%d' % n)   # x1..x_{n-1}
    A_tot = full_A(x_syms, n)
    return expand(A_tot.get(tuple(w), 0))

# --------------------------
# Example
# --------------------------
if __name__ == "__main__":
    # Example: Grothendieck polynomial for (1,5,3,2,4)
    print("(1,5,3,2,4) ->", grothendieck_polynomial((1,5,3,2,4), 5))

(1,5,3,2,4) -> 2*beta**2*x1**2*x2**2*x3*x4 + beta*x1**2*x2**2*x3 + beta*x1**2*x2**2*x4 + 3*beta*x1**2*x2*x3*x4 + 3*beta*x1*x2**2*x3*x4 + x1**2*x2*x3 + x1**2*x2*x4 + x1**2*x3*x4 + x1*x2**2*x3 + x1*x2**2*x4 + x1*x2*x3*x4 + x2**2*x3*x4
